In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
pd.options.display.max_colwidth = 3000

import warnings
warnings.filterwarnings(action='ignore')

In [8]:
earth = pd.read_csv("../data/[article]earthmoversmagazine.csv")
earth.sample(1)

,날짜,기사_제목,기사_url,기사_내용
154,15 December 2022,Cat announces collaboration with Luck Stone in USA,"https://www.earthmoversmagazine.co.uk/news/view,cat-announces-collaboration-with-luck-stone-in-usa_5693.htm","Caterpillar Inc has announced a collaboration with Luck Stone, the USA’s largest family-owned and operated producer of crushed stone, sand and gravel, to deploy Caterpillar’s autonomous solution to Luck Stone’s Bull Run Plant in Chantilly, Virginia. This will be Caterpillar’s first autonomous deployment in the aggregates industry and will expand the company’s autonomous truck fleet to include the 100-ton-class (90-tonne-class) Cat 777. Looking to accelerate autonomous solutions beyond mining, Caterpillar will implement its existing Cat MineStar Command for Hauling system at the Bull Run quarry, on a fleet of 777G trucks. This will allow Caterpillar to gain greater insights on quarry operations in order to tailor the next generation of autonomous solutions specific to quarry and aggregate applications. This project supports the acceleration of autonomous technology for operations with fewer mobile assets to allow a step change in safety and productivity, as currently experienced at large mining operations. “Luck Stone and Caterpillar’s partnership has been grounded in shared values for many years,” said Charlie Luck, President and CEO of Luck Companies. “Together we believe that safety, innovation and a commitment to people are critical, not only to propelling important projects like this, but to ensuring our focus on the future and all of the exciting possibilities technology affords our industry. Our collaboration will provide opportunities for associates to learn and grow, improve safety and result in production efficiencies. We are thrilled to partner with Caterpillar to achieve this ‘first’ for our industry.” <img alt="""" class=""inline-image"" height=""578"" src=""/ugc-1/1/6/0/Catluck1.jpg"" width=""800""/> Denise Johnson, Caterpillar Group President of Resource Industries said, “Caterpillar has a long-standing relationship with Luck Stone, and we look forward to working together to bring the demonstrated benefits of increased safety and productivity to the quarry industry. We’re excited to get in the dirt and work alongside Luck Stone’s innovative team, so we can learn how to scale our already proven mining solution for another industry.” The current autonomy solutions will be implemented in 2024. This project builds on Caterpillar’s long history in autonomy and automation. Caterpillar is recognised as the industry leader with the world’s largest fleet of autonomous haul trucks, now numbering more than 560 trucks. These trucks have travelled more than 187 million kilometres and autonomously moved more than 5.1 billion tonnes – most notably – all without a single zero lost-time injury."


In [35]:
# 불용어 파일 불러오기
stop_words = pd.read_csv("../data/stop_words.csv")
stop_words.tail(10)

,0
229,year
230,years
231,crane
232,cranes
233,-
234,2023
235,2022
236,2022
237,–
238,NaN


In [219]:
# 불용어로 쓸 단어 추가
stop_words_list = list(stop_words["0"])
stop_words_list.extend([" ", "s", "construction", "equipment", "machine", "volvo", "volvos", "ce"])
stop_words_list

['a',
 'but',
 'during',
 'hows',
 "it's",
 'said',
 'this',
 'we’re',
 'who’ve',
 'about',
 'by',
 'each',
 'however',
 'it’s',
 'says',
 'those',
 "we've",
 'whove',
 'above',
 'can',
 'either',
 'i',
 'its',
 'see',
 'through',
 'we’ve',
 'will',
 'across',
 "can't",
 'for',
 "i'd",
 "let's",
 'she',
 'to',
 'weve',
 'with',
 'after',
 'can’t',
 'from',
 'i’d',
 'let’s',
 "she'd",
 'too',
 'were',
 'within',
 'all',
 'cant',
 'given',
 "i'll",
 'lets',
 'she’d',
 'towards',
 'what',
 'without',
 'along',
 'cannot',
 'had',
 'i’ll',
 'may',
 'shed',
 'under',
 "what's",
 "won't",
 'also',
 'could',
 'has',
 "i'm",
 'me',
 "she'll",
 'until',
 'what’s',
 'won’t',
 'am',
 "couldn't",
 'have',
 'i’m',
 'more',
 'she’ll',
 'us',
 'whats',
 'would',
 'an',
 'couldn’t',
 'having',
 'im',
 'most',
 'shell',
 'use',
 'when',
 "wouldn't",
 'and',
 'couldnt',
 'he',
 "i've",
 'much',
 'should',
 'used',
 "when's",
 'wouldn’t',
 'any',
 'did',
 "he'd",
 'i’ve',
 'must',
 'since',
 'uses',
 'whe

In [148]:
# 추가 된 단어 포함하여 update 하기
pd.Series(stop_words_list).to_csv("../data/stop_words.csv", index=False)

## volvo 단어 빈도 추출해보기

In [46]:
volvo = pd.read_csv("../data/volvo_newsscraping.csv")
display(volvo.columns)
volvo.sample(1)

Index(['뉴스구분', '날짜', '제목', '요약', '본문'], dtype='object')

,뉴스구분,날짜,제목,요약,본문
32,By Press information,5/31/2022,CHANGE STARTS HERE - VOLVO DAYS 2022,"Volvo Construction Equipment (Volvo CE) will use the return of its iconic Volvo Days to highlight its purpose driven message of industry transformation. Using the theme of its recently launched 'Change Starts Here' campaign, the event showcases the company's leading solutions in the areas of sustainability, electromobility and services.","Highlighting the best of its products, services and people, Volvo Days launches today (May 31) on home ground in Eskilstuna, Sweden, with its most impressive line-up yet including the industry's widest range of electric machines for a more sustainable construction towards net zero emissions.\r\nAnd for those who cannot attend the three-week program in person, Volvo CE is offering for the very first time a fully-immersive digital experience that will ensure anyone — wherever they are in the world — can still be a part of the excitement with a new virtual platform being made available in the coming days.\r\nMelker Jernberg, Head of Volvo CE, says: ""As a world leading manufacturer of construction solutions we have the power to make a difference and it starts with us. We want to drive change in our own industry towards net zero emissions but also support our customers to decarbonize their operations.\r\n""Volvo Days is the perfect opportunity to show how we are achieving these ambitions with ready-today sustainable power solutions. It is a proud moment for me to meet our customers and partners, together with our passionate employees. Through close cooperation in the value chain, we can make this change happen.""\r\nCHANGE STARTS WITH ACTION\r\nThrough a series of product demonstrations and tours — both physical and virtual — guests can experience the full line-up of Volvo's electric compact machines and charging solutions as well as the company's largest electric excavator the EC230 Electric, currently being piloted in Norway. Also on display is the allnew L200H High Lift wheel loader promoting greater productivity and fuel efficiency, and the upgraded L350H wheel loader for superior strength and durability.\r\nAlso taking center stage are Volvo CE's ever-expanding range of productivity and uptime services — increasing in importance as the industry transitions from a machine-only perspective to site optimal service solutions. These include the recently launched C02 Reduction Program, offering tailored plans to cut carbon emissions, the Efficient Load Out solution, designed to make every load count with a 95-99% fill rate, the EMMA electric charging application, Connected Map and other digital solutions set to come to the market.\r\nSTRENGTH IN PARTNERSHIP\r\nVolvo's new demo and test track for autonomous solutions, designed in conjunction with Volvo Autonomous Solutions, is also being unveiled to the public for the first time at Volvo Days, enabling a test and demonstration site for battery electric and autonomous transport solutions to make construction sites safer, more efficient and more sustainable. Volvo Trucks, also part of Volvo Group, is another welcome addition to the line-up with its own electric truck present at the show.\r\nCarl Slotte, Head of Sales Region Europe, says: ""This legendary event has always been a proud showcase of our ability to match customer value and increased productivity with more sustainable solutions. And now for th..."


### volvo 데이터 정제하기
- 혜민님 꺼 참조
    - 데이터 정제
    - 문장 파생변수
    - 단어 파생변수
    - 불용어처리

In [97]:
volvo['날짜'] = pd.to_datetime(volvo['날짜'])
# \r\n 기호 제거
volvo = volvo.replace("\r\n", " ", regex=True)
# 특수 기호 제거 후 단어로 만들기
volvo["words"] = volvo["본문"].replace("\"|\–|\-|\(|\)|\|\:|\,", "", regex=True).map(lambda x : x.split(" "))

In [132]:
# 's 를 제거 하고싶다. 근데 volvo 기사들은 ’ 이 기호를 쓴다. 
volvo["정제본문"] = volvo["본문"].replace("’s", "", regex=True)
# 잡다한 특수기호 제거
volvo["정제본문"] = volvo["정제본문"].replace("\“|\”|\‘|\’|&|\"|\–|\-|\(|\)|\:|\,", "", regex=True)
# 띄어쓰기 두칸짜리 제거
volvo["정제본문"] = volvo["정제본문"].replace(" +", " ", regex=True)
# 소문자로 변경
volvo["정제본문"] = volvo["정제본문"].str.lower()

In [133]:
# 문장 단위 파생변수 만들기
volvo["문장"] = volvo["정제본문"].map(lambda x : x.split(". "))
# 문장갯수 뽑아보기
volvo["문장"].str.len()[:5]

0    21
1     4
2    23
3    20
4    25
Name: 문장, dtype: int64

In [172]:
# 불용어 처리 전 단어 파생변수 만들기
volvo["단어"] = volvo["문장"].map(lambda x : " ".join(x)).map(lambda x : x.split(" "))
volvo["단어"][:3]

0                                                                                         [in, q1, 2023, volvo, ce, has, secured, good, profitability, and, a, boost, to, its, operating, margin, with, continued, demand, in, large, infrastructure, and, construction, projects, particularly, in, north, america, and, europe, while, increasing, its, own, investments, into, the, biggest, technological, shift, in, the, industry, history, compared, to, the, same, period, last, year, volvo, ce, is, reporting, a, strong, boost, in, sales, of, 17%, in, europe, and, 37%, in, north, america, this, is, more, than, compensating, for, a, dip, in, sales, in, asia, and, south, america, where, investment, levels, have, slowed, down, with, the, exception, of, china, activity, in, the, construction, ...]
1                                                                            [kamel, who, has, held, the, position, of, acting, head, of, operations, since, last, september, when, jean, marc, des, courieres

### 불용어처리

In [173]:
# 불용어 처리 함수
def stop_words_out(words):
    result = []
    for w in words:
        if w not in stop_words_list:
            result.append(w)
    return " ".join(result)

In [178]:
# 불용어 처리한 단어 모음 (str)
volvo["clear_words_str"] = volvo["단어"].map(stop_words_out)
# 불용어 처리한 단어의 리스트
volvo["clear_words"] = volvo["clear_words_str"].map(lambda x: x.split(" "))

In [196]:
# 전처리 및 불용어 처리 한 df 를 저장해주기
volvo.to_csv("../data/clear/volvo_clear.csv", index=False)

### 6개월동안 등장한 단어 빈도
- 2022-11-01 이후로 생각하자

In [192]:
volvo_6month = volvo[["날짜", "clear_words"]].loc[volvo["날짜"] >= "2022-11-01"]
# 2022-11-01 이후 기사 수
volvo_6month.shape[0]

23

In [187]:
# 단어 하나하나 등장 갯수를 세서 dictionary 로 만들기
histogram_6month = {}  

for words in volvo_6month["clear_words"]: 
    for word in words:
        histogram_6month[word] = histogram_6month.get(word, 0) + 1 

print(histogram_6month)

{'q1': 4, 'secured': 1, 'good': 7, 'profitability': 2, 'boost': 4, 'operating': 10, 'margin': 3, 'continued': 10, 'demand': 8, 'large': 10, 'infrastructure': 10, 'projects': 8, 'particularly': 2, 'north': 25, 'america': 27, 'europe': 16, 'increasing': 4, 'own': 9, 'investments': 7, 'biggest': 4, 'technological': 2, 'shift': 5, 'industry': 31, 'history': 2, 'compared': 3, 'same': 11, 'period': 1, 'last': 12, 'reporting': 3, 'strong': 4, 'sales': 23, '17%': 1, '37%': 1, 'compensating': 1, 'dip': 1, 'asia': 3, 'south': 9, 'investment': 14, 'levels': 5, 'slowed': 3, 'down': 6, 'exception': 1, 'china': 8, 'activity': 9, 'markets': 19, 'driven': 4, 'primarily': 2, 'ongoing': 4, 'mining': 1, 'benefits': 10, 'commodity': 2, 'prices': 2, 'melker': 9, 'jernberg': 9, 'president': 13, 'robust': 1, 'like': 12, 'kind': 2, 'quarter': 8, 'course': 1, 'important': 15, 'testament': 1, 'great': 5, 'products': 11, 'service': 21, 'solutions': 51, 'continue': 11, 'bring': 3, 'market': 22, 'results': 4, 'all

In [188]:
# dictionary 의 데이터프레임화
data = {"word":histogram_6month.keys(), "count":histogram_6month.values()}
histogram_df = pd.DataFrame.from_dict(data)
histogram_df

,word,count
0,q1,4
1,secured,1
2,good,7
3,profitability,2
4,boost,4
...,...,...
2080,receive,1
2081,alexander,1
2082,lemke,1
2083,aebi,1


In [193]:
# 정렬
histogram_df = histogram_df.sort_values(by="count", ascending=False)
histogram_df.head()

,word,count
187,electric,150
167,customers,56
203,new,53
65,solutions,51
228,site,50


In [194]:
histogram_df.to_csv("../data/count/volvo_words_counts.csv", index=False)

# Caterpillar 단어 빈도 추출

In [202]:
cat = pd.read_csv("../scrapping/Caterpillar/Caterpillar.csv")
display(cat.columns)
cat.sample(1)

Index(['Date', 'Title', 'Article'], dtype='object')

,Date,Title,Article
88,2021-08-04,"Caterpillar expands Cat® Command remote-control operation to excavator line, enhancing safe machine operation","New Cat® Command for Excavating provides remote-control operation for select Cat excavators. Removing the operator from the machine in potentially hazardous operating environments, Command for Excavating enhances safe machine operation, while reducing operator fatigue to increase productivity. Six Cat excavators spanning the 20 to 40 metric ton class sizes – Cat 320, 323, 326, 330, 336 and 340 models – can now be equipped with either line-of-sight or non-line-of-sight remote operating configurations. Cat excavators are made Command-ready through a dealer-installed field kit for new or existing fleet machines. Providing excellent visibility, the kit features multiple cameras for non-line-of-site operation from the Command station that offer a view of the area surrounding the machine plus the ability to stream on-site video feeds. Alongside the cameras, indicator lights, microphone, wireless receiver, and antenna are all mounted on top of the cab to minimize the risk of damage. Deeply integrated with the excavator’s electronics, Command controls allow users to experience the same machine response as if they were operating from inside the cab, allowing a high level of productivity to be maintained from a safe distance. Machine technology features such as Grade Assist, Swing Assist and E-Fence can also be set, activated and deactivated remotely without the need to be set manually from inside the cab. Excavators are efficiently switched between remote and manual operation via a ground-level switch. With users comfortably controlling dig, lift and tracking functions from a safe location, Command for Excavating enables production to restart immediately following disruptive processes, such as blasting in quarry applications. It also enhances safety and improves machine productivity on a range of projects like operating in unstable underfoot conditions, brownfields, shore work, demolition and site decommissioning, and stevedoring. Flexible, precise control Command for Excavating offers a solution to workforce shortages by opening opportunities to those with physical limitations that prevent them from climbing into the equipment, attracting a new generation of worker and increasing the longevity of experienced operators. Removing the operator from the cab also offers training advantages for newer operators. The Command system includes two different remote operating options. Ideal for temporary or emergency remote operation with no on-site communications infrastructure required, the Command console offers line-of-site machine control through a lightweight, compact console, supported by a comfortable shoulder harness. Offering machine control from up to 437 yd (400 m) away, the console controls all excavator functions with virtually no response delay. Built-in safety features stop all excavator movements if the remote shutdown switch is pressed, wireless communication is lost, or the console is tilted more than 45 degrees from normal operating position. ..."


### CAT 전처리

In [207]:
cat['날짜'] = pd.to_datetime(cat['Date'])
# \r\n 기호 제거
cat = cat.replace("\r\n", " ", regex=True)

In [210]:
# 's 를 제거 하고싶다. 근데 volvo 기사들은 ’ 이 기호를 쓴다. 
cat["clean_rticle"] = cat["Article"].replace("’s", "", regex=True)
# 잡다한 특수기호 제거
cat["clean_rticle"] = cat["clean_rticle"].replace("\“|\”|\‘|\’|&|\"|\–|\-|\(|\)|\:|\,|\®", "", regex=True)
# 띄어쓰기 두칸짜리 제거
cat["clean_rticle"] = cat["clean_rticle"].replace(" +", " ", regex=True)
# 소문자로 변경
cat["clean_rticle"] = cat["clean_rticle"].str.lower()

In [212]:
# 문장 단위 파생변수 만들기
cat["sentence"] = cat["clean_rticle"].map(lambda x : x.split(". "))
# 문장갯수 뽑아보기
cat["sentence"].str.len()[:5]

0    31
1    51
2     7
3    33
4    53
Name: sentence, dtype: int64

In [214]:
# 불용어 처리 전 단어 파생변수 만들기
cat["word"] = cat["sentence"].map(lambda x : " ".join(x)).map(lambda x : x.split(" "))
cat["word"][:3]

0                           [infused, with, industryleading, technology, the, durable, cat, d10, dozer, burns, less, fuel, increases, productivity, and, improves, uptime, featuring, a, stator, clutch, torque, converter, and, loadsensing, hydraulics, the, new, design, is, up, to, 6%, more, efficient, than, the, cat, d10t2, its, combination, of, improved, technology, longer, component, life, extended, oil, changes, and, increased, serviceability, reduces, maintenance, and, repair, costs, up, to, 8%, helping, the, new, d10, deliver, industryleading, low, cost, of, ownership, the, new, d10, is, powered, by, the, cat, c27, engine, which, offers, aftertreatment, solutions, to, meet, u.s, epa, tier, 4, final/eu, stage, v, as, well, as, tier, 2, equivalent, ...]
1       [expanding, the, product, line, the, new, cat, rm600, and, rm800, reclaimer/stabilizers, offer, the, power, performance, and, productivity, needed, to, efficiently, complete, fulldepth, reclamation, and, soil, stabilization, pr

### 불용어 처리

In [222]:
# 불용어 처리해줄 단어 추가 및 저장
stop_words_list.extend(["caterpillar", "cat"])
pd.Series(stop_words_list).to_csv("../data/stop_words.csv", index=False)
stop_words_list[-5:]

['ce', 'caterpillar', 'cat', 'caterpillar', 'cat']

In [223]:
# 불용어 처리 함수
def stop_words_out(words):
    result = []
    for w in words:
        if w not in stop_words_list:
            result.append(w)
    return " ".join(result)

In [227]:
# 불용어 처리한 단어 모음 (str)
cat["clear_words_str"] = cat["word"].map(stop_words_out)
# 불용어 처리한 단어의 리스트
cat["clear_words"] = cat["clear_words_str"].map(lambda x: x.split(" "))
cat["clear_words"][:2]

0                                                        [infused, industryleading, technology, durable, d10, dozer, burns, less, fuel, increases, productivity, improves, uptime, featuring, stator, clutch, torque, converter, loadsensing, hydraulics, new, design, up, 6%, efficient, d10t2, combination, improved, technology, longer, component, life, extended, oil, changes, increased, serviceability, reduces, maintenance, repair, costs, up, 8%, helping, new, d10, deliver, industryleading, low, cost, ownership, new, d10, powered, c27, engine, offers, aftertreatment, solutions, meet, u.s, epa, tier, 4, final/eu, stage, v, well, tier, 2, equivalent, emissions, standards, meeting, needs, global, market, maximizing, material, moved, per, liter, fuel, c27, switches, power, settings, based, travel, direction, offer, up, 20%, power, reverse, reducing, cycle, times, addition, delivering, ...]
1    [expanding, product, line, new, rm600, rm800, reclaimer/stabilizers, offer, power, performance, produc

In [228]:
# 전처리 및 불용어 처리 한 df 를 저장해주기
cat.to_csv("../data/clear/cat_clear.csv", index=False)

### 6개월 동안 등장한 단어 빈도
- 2022-11-01 이후로 생각하자

In [229]:
cat_6month = cat[["Date", "clear_words"]].loc[cat["Date"] >= "2022-11-01"]
# 2022-11-01 이후 기사 수
cat_6month.shape[0]

31

In [231]:
# 단어 하나하나 등장 갯수를 세서 dictionary 로 만들기
histogram_6month = {}  

for words in cat_6month["clear_words"]: 
    for word in words:
        histogram_6month[word] = histogram_6month.get(word, 0) + 1 

print(histogram_6month)

{'infused': 1, 'industryleading': 6, 'technology': 53, 'durable': 5, 'd10': 13, 'dozer': 13, 'burns': 1, 'less': 19, 'fuel': 66, 'increases': 10, 'productivity': 51, 'improves': 20, 'uptime': 15, 'featuring': 8, 'stator': 4, 'clutch': 7, 'torque': 20, 'converter': 4, 'loadsensing': 3, 'hydraulics': 4, 'new': 192, 'design': 50, 'up': 72, '6%': 2, 'efficient': 12, 'd10t2': 2, 'combination': 8, 'improved': 22, 'longer': 18, 'component': 10, 'life': 46, 'extended': 17, 'oil': 35, 'changes': 11, 'increased': 21, 'serviceability': 2, 'reduces': 16, 'maintenance': 55, 'repair': 5, 'costs': 29, '8%': 4, 'helping': 14, 'deliver': 22, 'low': 5, 'cost': 13, 'ownership': 4, 'powered': 5, 'c27': 3, 'engine': 70, 'offers': 44, 'aftertreatment': 3, 'solutions': 23, 'meet': 16, 'u.s': 5, 'epa': 7, 'tier': 10, '4': 15, 'final/eu': 2, 'stage': 8, 'v': 7, 'well': 17, '2': 5, 'equivalent': 5, 'emissions': 9, 'standards': 6, 'meeting': 2, 'needs': 12, 'global': 36, 'market': 6, 'maximizing': 5, 'material':

In [234]:
# dictionary 의 데이터프레임화
data = {"word":histogram_6month.keys(), "count":histogram_6month.values()}
histogram_df = pd.DataFrame.from_dict(data)
# 정렬
histogram_df = histogram_df.sort_values(by="count", ascending=False)
histogram_df

,word,count
20,new,192
108,operator,120
207,system,73
22,up,72
148,standard,72
...,...,...
1990,bonus,1
1980,nate,1
1978,disrupters,1
1977,innovations,1


In [235]:
histogram_df.to_csv("../data/count/cat_words_counts.csv", index=False)

## Earthmovers

In [237]:
earth = pd.read_csv("../data/[article]earthmoversmagazine.csv")
display(earth.columns)
earth.sample(1)

Index(['날짜', '기사_제목', '기사_url', '기사_내용'], dtype='object')

,날짜,기사_제목,기사_url,기사_내용
73,09 March 2023,Yanmar ViO38 outstanding in its field,"https://www.earthmoversmagazine.co.uk/news/view,yanmar-vio38-outstanding-in-its-field_5850.htm","If there is one thing that Yanmar Compact Equipment EMEA (Yanmar)’s ViO38 mini-excavator is guaranteed to offer, it’s versatility. Digging out an extensive pond in the morning, then trimming and felling roadside trees in the afternoon? That’s a typical day in the life of a ViO38. Just ask Martin Parish Contracting (MPC), a multifaceted forestry and agricultural contractor in Chulmleigh, in deepest rural North Devon. Agriculture is fundamental to Devon and its rural communities, with farming being the mainstay of the local landscape for centuries. While MPC’s owner – Martin Parish – launched his business as recently as 2016, he has been working with machinery since his teens. MPC, as is often found in the countryside, needs to be a jack of all trades, one day making household driveway alterations, to next day landscaping lakes. For these tasks and many more besides, MPC relies on a single Yanmar ViO38 mini excavator, fitting the zero-tailswing unit with various attachments and taking pleasure in regularly testing the breadth of its capabilities: “As a small business, having a multitask machine that can do everything is very important to us. We fit our Yanmar mini excavator with a variety of different buckets for landscaping and groundworks projects; a cutter for hedge and tree trimming; a wire roller and post driver for erecting our fences; a grapple for lifting timber. So many of our tasks rely on the performance of our ViO38. It’s also so easy to operate, and its cabin is really comfortable and surprisingly spacious given the machine’s compact size. This is a huge advantage given how much time we spend working in it every day.” Hydraulic Quick Coupler enables the operator to hydraulically open and close the couplers from their seat at the flick of a switch. The ViO38’s spacious and airtight cabin also features foldable pedals for more operator space, improved ergonomics, and optional air conditioning to maximise comfort around the clock. <img alt="""" class=""inline-image"" height=""450"" src=""/ugc-1/1/6/0/Yanmar CE MPC visual 3_edit.jpg"" width=""800""/> Tough terrain Much of MPC’s daily jobs revolve around forest management, where Martin and his team will head into Devon’s dense woodland to complete drainage and maintenance work, along with other general forestry activities. These woodlands can often be found on steep valley slopes, and having a robust, reliable unit that can maintain surefootedness while working on difficult terrain is vital, explained Martin. “The forestry work is always challenging, as nine times out of 10 the conditions are steep, wet and in the middle of nowhere. We certainly put the ViO38 mini excavator through its paces with such projects, but it has stood up to the test and proven itself capable of continuing to perform reliably and efficiently.” Yanmar has combined many clever design features to make the ViO38 very stable, including a low centre of gravity and long undercarriage to ensure a smoother, safer ride. The unit i..."


In [238]:
earth.sample(3)

,날짜,기사_제목,기사_url,기사_내용
132,13 January 2023,Hyundai signs up The Digger Girl for 2023,"https://www.earthmoversmagazine.co.uk/news/view,hyundai-signs-up-the-digger-girl-for-2023_5735.htm","Hyundai Construction Equipment Europe (HCEE) has announced an official partnership with Amy Underwood, ‘The Digger Girl’ who will act as a brand ambassador for Hyundai CEE in 2023. As one of the industry’s most popular social media content creators, Amy has been brought on board by HCEE to raise the profile of the Hyundai brand amongst fans and fellow industry professionals, producing exclusive content and giving her followers a unique, behind the scenes insight into Hyundai’s European operations and its latest range of A-Series machines. Amy will continue to have full editorial control of her content but through her partnership with HCEE she will receive exclusive opportunities to access Hyundai’s full range of machines and will be able to create content at Hyundai’s European construction headquarters in Tessenderlo, Belgium. Dr Hubertus Muenster, Director Sales & Marketing at Hyundai Construction Equipment Europe said, “Hyundai’s mantra this year is ‘Think Further, Think Future’ – a holistic commitment to becoming more innovative and future-focused across our entire operation. As part of this, we recognised the importance of developing new ways for our brand to connect with new audiences, and working with social media content creators like Amy to strengthen our online presence is a fantastic opportunity to achieve this.” <img alt="""" class=""inline-image"" height=""533"" src=""/ugc-1/1/6/0/TDG2.jpg"" width=""800""/> Emma Dutton, Senior Marketing Manager at Hyundai Construction Equipment Europe, continued, “We have long been a supporter of Amy’s fantastic work, raising the profile of women in construction. She is a well-respected, high-profile figure in our industry and we believe the work we can do together will be really exciting. “Amy has always been extremely positive about her experiences of working with Hyundai machines and her recent trip out to our stand at Bauma was a great opportunity for her to get under the skin of Hyundai and to really experience our Think Further, Think Future ethos,” Emma continued. “We are delighted to welcome Amy on board as our 2023 brand ambassador and look forward to creating great content with her this year. Amy Underwood, The Digger Girl said, “I’m excited and proud to partner with Hyundai – a brand I have loved to use and advocated for over the years. This relationship will give me access to unique opportunities and brand new, top-of-the-line machines to try out. I can’t wait to share the photos and videos of this with those who have been following my journey. Our deal also helps me encourage more girls to get into construction and show that a successful career in this industry can also go alongside being a mum – though it’s not easy!” To find out more about The Digger Girl, check out @the_digger_girl on Instagram or TikTok. Amy is also starting a YouTube channel which will feature regular videos to document her 2023 journey – you can subscribe here: https://www.youtube.com/@the_digger_girl."
127,18 January 2023,Kilmac operator chooses mighty Hyundai HX85A,"https://www.earthmoversmagazine.co.uk/news/view,kilmac-operator-chooses-mighty-hyundai-hx85a_5746.htm","“These will be the first three Hyundais of many,” explained Bert Hope of Tayside-based Civil Engineering and Groundworks contractor, Kilmac Ltd, who has recently taken delivery of three new, 8.5-tonne HX85A compact excavators. The company covers projects across the whole of Scotland including a huge development to introduce wider foot and cycle paths on the Dundee to Aberdeen Road – a project that supports the nationwide Active Travel initiative. Kilmac was looking to expand its fleet of excavators with something strong, good value for money and of an excellent build quality. The new 8.5-tonne A-Series machine proved to be the perfect fit – so goo

# 코마쯔

In [240]:
komaz = pd.read_csv("../data/komatsu_newsscraping.csv")
display(komaz.columns)
komaz.sample(1)

Index(['날짜', '제목', '본문'], dtype='object')

,날짜,제목,본문
26,"Sep. 21, 2022",- Sustainable growth through structural reform- Komatsu sells 4 production facilities in China,"Komatsu Ltd. (hereafter “Komatsu”) (President & CEO: Hiroyuki Ogawa) through its subsidiary, Komatsu Mining Corp. (President & CEO: Peter Salditt) today announced the sale of four facilities which are a part of its underground soft rock business mining equipment in China. The facilities in Qingdao City, Shandong; Huainan City, Anhui; and Jixi City and Jiamusi City, Heilongjiang; were purchased by Jixi Gengchen Motor Manufacturing Co., Ltd. (Heilongjiang) and Nanjing Luhe Coal Mine Machinery Co., Ltd. (Jiangsu).With this divestment there is no impact on the supply and servicing of underground soft rock products for customers in China. Komatsu estimates this transaction will have minimal effect on its consolidated business results.The buyers for the four sold properties are both Chinese companies that manufacture and sell equipment and electronic components for mining of underground coal. Through this purchase, the two companies aim to further expand their business in China.Komatsu is working to provide its customers increasingly sustainable ways to extract essential minerals worldwide. The company has continued to adjust its business portfolio flexibly in response to changes in operating environments and market trends. With this sale supporting ongoing structural reforms, Komatsu continues to build on a sustainably profitable product portfolio, as well as continued focus on growth. No : 0046（3166）Corporate Communications DepartmentSustainability Promotion DivisionKomatsu Ltd.tel: +81-(0)3-5561-2616mail: JP00MB_cc_department@global.komatsu*The information may be subject to change without notice."


In [241]:
komaz.sample(3)

,날짜,제목,본문
52,"Jan. 31, 2022",Komatsu Agrees to Make Komatsu (Changzhou) Construction Machinery a Wholly Owned Subsidiary,"Komatsu Ltd. (hereafter ""Komatsu"") (President & CEO: Hiroyuki Ogawa) has agreed to dissolve the equity participation of SINOMACH Changlin Co., Ltd. (hereafter ""Changlin""), based in Jiangsu, China, and Sumitomo Corporation (hereafter ""Sumitomo"") in Komatsu (Changzhou) Construction Machinery Corp. [hereafter ""Komatsu (Changzhou)""], based in Jiangsu, China, a joint-venture company with Changlin and Sumitomo.In 1995, Komatsu established Komatsu (Changzhou), a joint-venture manufacturer of wheel loaders in Changzhou, Jiangsu Province, with local manufacturers of construction equipment, such as Changlin. Later, Komatsu (Changzhou) added hydraulic excavators and other equipment to production and has supplied its products mainly in China.After discussing with Changlin the future directions of the business, in light of changes in the business environment for the last few years, Komatsu and Changlin have agreed to promote their respective businesses independently and dissolve Changlin’s equity participation in Komatsu (Changzhou). In line with the agreement with Changlin to dissolve its equity participation, Komatsu and Sumitomo have also agreed to dissolve Sumitomo’s equity participation in Komatsu (Changzhou). In addition, Komatsu and Sumitomo have already agreed to dissolve Sumitomo’s equity participation in Komatsu Shantui Construction Machinery Co., Ltd.Together with the partners, Komatsu will proceed with the process as required in FY2021, ending March 31, 2022, and believes that there will be no adverse effects on customers with respect to product delivery. Komatsu also anticipates that the impact of this termination on its consolidated business results will be minimal. <Outline of the Komatsu (Changzhou) Construction Machinery>Name : Komatsu (Changzhou) Construction Machinery Corp.Located : Changzhou, Jiangsu, ChinaRepresentative : Quanwang Zhang, Chairman of the BoardLine of business: Manufacture of construction equipmentCapital Stock : US $ 41 millionEstablished : May 1995Equity : 85% by Komatsu (China) Ltd., 8% by Changlin, and 7% by SumitomoNo : 0088(3103)Corporate Communications DepartmentSustainability Promotion DivisionKomatsu Ltd.tel: +81-(0)3-5561-2616mail: JP00MB_cc_department@global.komatsu*The information may be subject to change without notice."
12,"Feb.15,2023",Relief Aid to Earthquake Damages in Turkey and Syria,"Komatsu Ltd. (President and CEO: Hiroyuki Ogawa) (hereinafter “Komatsu”) expresses its deepest condolences to those who lost their lives in the recent earthquake that occurred in southeastern Turkey. Komatsu also extends its deepest sympathies to all those affected by the earthquake.In cooperation with its local distributor Marubeni Dağıtım ve Servis A.Ş. (hereinafter “MDS”), a wholly owned subsidiary of Marubeni Corporation, Komatsu will lend construction equipment needed for relief efforts to the affected areas free of charge and dispatch operators and service mechanics to assist the relief efforts, which should amount to JPY 20 million equivalent. Komatsu will donate JPY 10 million through the Japanese Red Cross Society (total support of 30 million yen).Since establishing an office in Turkey in 1975, Komatsu has been engaging its locally-anchored business in Turkey, a major market in the Middle East region. Komatsu will continue to provide material assistance to help the affected areas recover as quickly as possible while monitoring the conditions with MDS. No : 0086(3206)Corporate Communications DepartmentSustainability Promotion DivisionKomatsu Ltd.tel: +81-(0)3-5561-2616mail: JP00MB_cc_department@global.komatsu*The information may be subject to change without notice."
48,"March 25, 2022",Komatsu Opens Parts Center in Australia: Improving customer satisfaction and optimizing operations,"Komatsu Ltd. (hereafter “Komatsu”) (President and CEO: Hiroyuki Ogawa) has opened the n

# 날짜 데이터프레임

In [246]:
display(cat.columns)
cat.info()

Index(['Date', 'Title', 'Article', '날짜', 'words', 'clean_rticle', 'sentence',
       'word', 'clear_words_str', 'clear_words'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Date             100 non-null    object        
 1   Title            100 non-null    object        
 2   Article          100 non-null    object        
 3   날짜               100 non-null    datetime64[ns]
 4   words            100 non-null    object        
 5   clean_rticle     100 non-null    object        
 6   sentence         100 non-null    object        
 7   word             100 non-null    object        
 8   clear_words_str  100 non-null    object        
 9   clear_words      100 non-null    object        
dtypes: datetime64[ns](1), object(9)
memory usage: 7.9+ KB


In [248]:
cat.isnull().sum()

Date               0
Title              0
Article            0
날짜                 0
words              0
clean_rticle       0
sentence           0
word               0
clear_words_str    0
clear_words        0
dtype: int64

In [252]:
test_df = cat[["날짜","clear_words"]][:2]
test_df

,날짜,clear_words
0,2023-05-22,"[infused, industryleading, technology, durable, d10, dozer, burns, less, fuel, increases, productivity, improves, uptime, featuring, stator, clutch, torque, converter, loadsensing, hydraulics, new, design, up, 6%, efficient, d10t2, combination, improved, technology, longer, component, life, extended, oil, changes, increased, serviceability, reduces, maintenance, repair, costs, up, 8%, helping, new, d10, deliver, industryleading, low, cost, ownership, new, d10, powered, c27, engine, offers, aftertreatment, solutions, meet, u.s, epa, tier, 4, final/eu, stage, v, well, tier, 2, equivalent, emissions, standards, meeting, needs, global, market, maximizing, material, moved, per, liter, fuel, c27, switches, power, settings, based, travel, direction, offer, up, 20%, power, reverse, reducing, cycle, times, addition, delivering, ...]"
1,2023-05-04,"[expanding, product, line, new, rm600, rm800, reclaimer/stabilizers, offer, power, performance, productivity, needed, efficiently, complete, fulldepth, reclamation, soil, stabilization, projects, versatile, machines, compete, high, end, reclaimer/stabilizer, size, class, allnew, adjustable, height, design, capable, crossslope, cutting, powerful, turbocharged, c18, engine, gives, rm600, up, 17%, power, rm500b, rm800, delivers, up, 42%, power, rm500b, up, 23%, power, rm600, reclaimer/stabilizer, models, adopt, key, rotor, system, components, cold, planers, dependable, cutting, mixing, featuring, belt, driven, rotor, drive, system, capable, flushcutting, right, side, new, system, k, rotor, same, cold, planer, fastenerfree, dualtaper, toolholder, base, block, components, deliver, superior, material, quality, range, projects, exit, returntocut, features, simplify, operation, ...]"


In [285]:
def label_date(df):
    result = pd.DataFrame(columns=["date", "words"])
    for i in range(len(df)):
        temp = pd.DataFrame(df.iloc[i]["clear_words"], columns=["words"])
        temp["date"] = df.iloc[i]["날짜"]
        result = pd.concat([result, temp])
    
    return result

In [286]:
label_date(test_df)

,date,words
0,2023-05-22,infused
1,2023-05-22,industryleading
2,2023-05-22,technology
3,2023-05-22,durable
4,2023-05-22,d10
...,...,...
710,2023-05-04,found
711,2023-05-04,contacting
712,2023-05-04,dealer
713,2023-05-04,visiting


In [287]:
label_date(cat[["날짜","clear_words"]])

,date,words
0,2023-05-22,infused
1,2023-05-22,industryleading
2,2023-05-22,technology
3,2023-05-22,durable
4,2023-05-22,d10
...,...,...
273,2021-06-01,6229
274,2021-06-01,las
275,2021-06-01,vegas
276,2021-06-01,convention
